In [2]:
#Import required libraries
import gspread
import sys
#from oauth2client.service_account import ServiceAccountCredentials
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [4]:
sys.path.append('..')

In [5]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [6]:
sys.path.append('..\..')

In [7]:
#Import shared functions
#sys.path.append('..')
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [8]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [9]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [30]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_cw')

In [31]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [32]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [33]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [34]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [35]:
sql = 'select * from crowdsdb.dbo.tbl_cw_ambassador'

In [36]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [37]:
format_datetime(ambass_sql, 'encounter_timestamp')
format_datetime(ambass_sql, 'encounter_datetime')

In [38]:
sql_cols = list(ambass_sql.columns.values)

In [39]:
float_cols = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']
for c in float_cols:
    ambass_sql[c] = ambass_sql[c].astype(float)

In [40]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [27]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

In [28]:
creds = service_account.Credentials.from_service_account_file(cred_file).with_scopes(scope)

In [29]:
client = gspread.Client(creds)

In [19]:
sheet = client.open('_COMBINED Inter-Agency Parks Social Distancing Education Ambassador Report (Responses)')

In [20]:
ws = sheet.worksheet('RESPONSES')

In [23]:
ambass = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [24]:
ambass.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
1,6/17/2020 14:32:06,6/13/2020 14:09:00,Central Park,NaN,OATH,Anessa,Emanuel,Lorna,Mondesir,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,NaN
2,6/17/2020 14:30:24,6/17/2020 13:29:00,Central Park,NaN,OATH,Lorna,Mondesir,Anessa,Emmanuel,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,NaN
3,6/17/2020 14:01:54,6/17/2020 14:00:00,Owl's Head Park,NaN,DDC,Joseph,Stankus,NaN,NaN,NaN,...,NaN,NaN,Skate Park,10.0,No,No,NaN,Increase to 10 in the skatepark . Only a few w...,Brooklyn,NaN
4,6/17/2020 13:37:59,6/17/2020 13:35:00,Owl's Head Park,NaN,DDC,Joseph,Stankus,NaN,NaN,NaN,...,NaN,NaN,Basketball court,5.0,No,No,NaN,Working out on the courts . All were maintaini...,Brooklyn,NaN
5,6/17/2020 13:07:48,6/17/2020 13:07:00,Prospect Park,NaN,OATH,Alex,Kolodka,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,NaN


In [46]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [47]:
yesno_cols(ambass, yesno)

In [48]:
format_datetime(ambass, 'encounter_timestamp')
format_datetime(ambass, 'encounter_datetime')

In [49]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_comments,closed_amenity,closed_patroncount,closed_comments,borough,sd_pdcontact,closed_approach,closed_outcome,closed_pdcontact
1,06-15-2020 08:45:10,06-14-2020 18:33:00,Forest Park,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Queens,NaN,NaN,NaN,NaN
2,06-15-2020 07:45:42,06-13-2020 18:00:00,Tompkins Square Park,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,Gave out masks,NaN,NaN,NaN,Manhattan,NaN,NaN,NaN,NaN
3,06-15-2020 07:40:03,06-13-2020 17:26:00,Tompkins Square Park,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,Gave out masks,NaN,NaN,NaN,Manhattan,NaN,NaN,NaN,NaN
4,06-15-2020 07:35:59,06-13-2020 16:56:00,Tompkins Square Park,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,Manhattan,NaN,NaN,NaN,NaN
5,06-15-2020 07:33:49,06-13-2020 16:07:00,Tompkins Square Park,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,Manhattan,NaN,NaN,NaN,NaN


In [50]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [51]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [52]:
#ambass[ambass['site_id'].isnull()]['site_desc'].unique()

In [53]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-15-2020 08:45:10,06-14-2020 18:33:00,Q015,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1,06-15-2020 07:45:42,06-13-2020 18:00:00,M088,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
2,06-15-2020 07:40:03,06-13-2020 17:26:00,M088,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
3,06-15-2020 07:35:59,06-13-2020 16:56:00,M088,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,06-15-2020 07:33:49,06-13-2020 16:07:00,M088,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [54]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [55]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [56]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,06-15-2020 08:45:10,06-14-2020 18:33:00,Q015,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,I
1,06-15-2020 07:45:42,06-13-2020 18:00:00,M088,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
2,06-15-2020 07:40:03,06-13-2020 17:26:00,M088,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
3,06-15-2020 07:35:59,06-13-2020 16:56:00,M088,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
4,06-15-2020 07:33:49,06-13-2020 16:07:00,M088,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I


In [57]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [58]:
ambass_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,06-15-2020 08:45:10,06-14-2020 18:33:00,Q015,NaN,DEP,Jesus,La Torre,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1,06-15-2020 07:45:42,06-13-2020 18:00:00,M088,9th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
2,06-15-2020 07:40:03,06-13-2020 17:26:00,M088,8th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,Walking path,NaN,Gave out masks,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
3,06-15-2020 07:35:59,06-13-2020 16:56:00,M088,7th Street Avenue A gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,06-15-2020 07:33:49,06-13-2020 16:07:00,M088,7th Street Avenue B gave out masks,DEP,TRACEY,CORNISH,Fiona,Cummings,Ashmeed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [59]:
ambass_inserts.to_sql('tbl_cw_ambassador', engine, index = False, if_exists = 'append')

In [60]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [61]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1258,06-09-2020 11:34:18,05-30-2020 16:30:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1259,06-09-2020 11:34:18,05-30-2020 16:30:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1266,06-09-2020 11:34:14,05-28-2020 16:25:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1267,06-09-2020 11:34:14,05-28-2020 16:25:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1270,06-09-2020 11:34:13,05-27-2020 14:30:00,Q006,NaN,OATH,Jaurez,Davis,Samad,Newaz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [62]:
sql_update(ambass_updates, 'tbl_cw_ambassador', engine, ['encounter_timestamp', 'site_id'])